In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pymysql
import warnings
warnings.filterwarnings('ignore')

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [3]:
sql = '''
select tprm.order_create_time, tprm.order_number, tprm.purchase_amount, tprm.overdue_type, tprm.overdue_loss,
topc.policy_status, topc.money 投保保费, tprm.rent_received, topc.policy_url
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order_picc topc on topc.order_id=tprm.order_id
'''
df = query(sql)

In [4]:
# policy_status'投保状态 1、已投保  2、已退保'
df_tb = df[(df.policy_status==1)]
df_tb.loc[:, '下单月份'] = df_tb.order_create_time.dt.strftime('%Y-%m')
df_tb.loc[:, '投保公司'] = df_tb.policy_url.str.extract(r'(epicc?|chinahuanong?)')[0]
df_tb.loc[:, '逾期订单数'] = np.where(df_tb.overdue_type.isin(['M0', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6']), 1, 0)
df_tb.loc[:, '逾期订单成本'] = np.where(df_tb.逾期订单数==1, df_tb.purchase_amount, 0)
df_tb.loc[:, '逾期订单已收'] = np.where(df_tb.逾期订单数==1, df_tb.rent_received, 0)

In [5]:
df_tb_hn = df_tb[df_tb.投保公司=='chinahuanong']
df_tb_group = df_tb_hn.groupby('下单月份').agg({'order_number': 'count', '投保保费': 'sum', 'purchase_amount': 'sum', '逾期订单数': 'sum', '逾期订单已收': 'sum', '逾期订单成本': 'sum'}).rename(columns={'order_number': '投保订单数', 'purchase_amount': '投保订单成本'})
df_tb_group.loc[:, '逾期损失'] = df_tb_group.逾期订单成本-df_tb_group.逾期订单已收
df_tb_group.loc[:, '订单逾期率'] = (df_tb_group.逾期订单数/df_tb_group.投保订单数).map(lambda x: format(x, '.2%'))
df_tb_group.loc[:, '逾期损失率'] = (df_tb_group.逾期损失/df_tb_group.投保订单成本).map(lambda x: format(x, '.2%'))


In [6]:
df_tb_group = df_tb_group[['投保订单数', '投保保费', '投保订单成本', '逾期订单数', '逾期订单已收', '逾期损失', '订单逾期率', '逾期损失率']]
# with pd.ExcelWriter('F:/需求/奇哥需求/投保订单.xlsx', engine='xlsxwriter') as writer:
#     df_tb_group.to_excel(writer, sheet_name='投保订单汇总')

In [7]:
df_tb_group

,投保订单数,投保保费,投保订单成本,逾期订单数,逾期订单已收,逾期损失,订单逾期率,逾期损失率
下单月份,,,,,,,,
2023-11,1,119.99,9155.0,1,3249.37,5905.63,100.00%,64.51%
2023-12,162,18009.18,1343986.5,82,439141.99,232199.01,50.62%,17.28%
2024-01,341,38286.19,2754951.0,152,902973.27,325209.73,44.57%,11.80%
2024-02,147,15668.13,1119359.0,60,334392.73,123954.27,40.82%,11.07%
2024-03,373,40873.07,2903087.0,168,853258.74,456715.26,45.04%,15.73%
2024-04,210,22692.30,1614400.0,81,384354.63,240262.37,38.57%,14.88%
2024-05,528,57525.12,3837481.0,216,922543.50,656243.50,40.91%,17.10%
2024-06,1093,113678.87,7632566.0,393,1495462.07,1313700.93,35.96%,17.21%
2024-07,2735,291468.95,19718180.0,1045,4146988.22,3533806.78,38.21%,17.92%
